<a href="https://colab.research.google.com/github/uob-positron-imaging-centre/PEPT-Algorithms-RoPP/blob/main/FPI_RoPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a target="_blank"  href="https://github.com/uob-positron-imaging-centre/pept"><img src="https://github.com/uob-positron-imaging-centre/misc-hosting/blob/master/logo.png?raw=true" style="height:200px; display: block; margin-left: auto; margin-right: auto;"/></a>

# Interactive PEPT Analysis Examples using *the Feature Point Identification algorithm*

> [1] Wiggins C, Santos R, Ruggles A. A feature point identification method for positron emission particle tracking with multiple tracers. Nuclear Instruments and Methods in Physics Research Section A: Accelerators, Spectrometers, Detectors and Associated Equipment. 2017 Jan 21;843:22-8.

---

#### Copyright 2021 the `pept` developers
##### Jupyter Notebook authored by Andrei Leonard Nicusan and Dr. Kit Windows-Yule for the "PEPT: A Comparative Review" paper, commissioned by the Reports on Progress in Physics journal

Licensed under the GNU License, Version 3.0 (the "License").

---


# 1. Introduction

Positron emission particle tracking (PEPT) is a powerful technique allowing the non-invasive, three-dimensional tracking of one or more radioactive 'tracer' particles through particulate, fluid or multiphase systems. It allows particle or fluid motion to be tracked with sub-millimetre accuracy and sub-millisecond temporal resolution and, due to its use of highly-penetrating 511keV gamma rays, can be used to probe the internal dynamics of even large, dense, optically opaque systems <sup>[[2]](https://www.sciencedirect.com/science/article/pii/016890029390864E) [[3]](https://www.sciencedirect.com/science/article/pii/S0263876208003341) [[4]](https://aip.scitation.org/doi/abs/10.1063/1.4983046@rsi.2017.IMGP2017.issue-1)</sup>. In light of its versatility both in terms of the scales and materials of particles which can be tracked <sup>[[5]](https://www.sciencedirect.com/science/article/pii/S1672251507001455)[[6]](https://www.sciencedirect.com/science/article/pii/S0168900206005341)</sup>, and the sizes and geometries of the systems which can be imaged <sup>[[7]](https://www.sciencedirect.com/science/article/pii/S0168900209001880) [[8]](https://www.sciencedirect.com/science/article/pii/S0029549316000273)</sup> , the technique has wide-ranging applicability in diverse scientific, industrial and biomedical applications.

PEPT is performed by radioactively labelling a particle with a positron-emitting radioisotope such as Fluorine-18 ($^{18}\mathrm{F}$) or Gallium-68 ($^{68}\mathrm{Ga}$), and using the back-to-back gamma rays produced by electron-positron annihilation events in and around the tracer to triangulate its spatial position. Each detected gamma ray represents a **line of response (LoR)** .

## 1.1. This Jupyter Notebook

This interactive Jupyter Notebook illustrates the main processing steps employed by the Feature Point Identification (FPI) algorithm<sup>[1]</sup> for radioactive tracer tracking, as described in the Reports on Progress in Physics "PEPT: A Comparative Review" paper.

An [example dataset](https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv) is used from an experiment run at the University of Birmingham Positron Imaging Centre using the ADAC Forté by Matthew Herald. It consists of a single 1 mm diameter MCC particle activated with Fluorine-18 radioactive tracer material inside a bubbling fluidised bed. The fluidised bed was filled with 90% sand and 10% MCC; air was fed into the bottom of the bed at a rate of 37 litres per minute at 3.5 bar. This dataset was chosen for its high quality captured lines of response, with the tracer still depicting the random particle motion that is inherent to bubbling fluidised beds - and typical in Lagrangian particle tracking.

The [`pept`](https://github.com/uob-positron-imaging-centre/pept) Python library is used for initialising and visualising PEPT data. While not required *per se* for illustrating PEPT algorithms' processing steps, it significantly reduces the amount of repetitive code and visual noise, allowing the reader to focus on the main conceptual procedures.

## 1.2. Running Code Cells
Select any code cell and click on the (▶) sign in the top-left of the cell's frame to run its code (note that code cells have to be run in order when running for the first time).

In [14]:
# First install the `pept` library using pip, Python's package manager
!pip install pept

# 2. The FPI Algorithm

## 2.1. Read in Line of Response Data
In this initial step, we read in the "raw" LoR data from which our tracer positions are calculated. In this simple example, we load only a single sample (i.e. one particle location at one point in time). The LoRs corresponding to this sample are output as an image.

As discussed in the main text, varying the numbers of LoRs used to calculate a PEPT location can affect the quality of the final location measured. Try altering the value "nrows" below to see how this inluences your results.

In [15]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 50,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.000e+00, 0.000e+00, 1.687e+02, 6.000e+02, 1.428e+02],
       [1.000e-01, 0.000e+00, 1.676e+02, 6.000e+02, 3.139e+02],
       [1.000e-01, 0.000e+00, 4.100e+02, 6.000e+02, 2.401e+02],
       [2.000e-01, 0.000e+00, 2.962e+02, 6.000e+02, 4.525e+02],
       [2.000e-01, 0.000e+00, 1.151e+02, 6.000e+02, 3.534e+02],
       [2.000e-01, 0.000e+00, 1.322e+02, 6.000e+02, 2.661e+02],
       [2.000e-01, 0.000e+00, 3.735e+02, 6.000e+02, 1.310e+02],
       [3.000e-01, 0.000e+00, 1.115e+02, 6.000e+02, 3.534e+02],
       [4.000e-01, 0.000e+00, 2.094e+02, 6.000e+02, 2.808e+02],
       [4.000e-01, 0.000e+00, 2.749e+02, 6.000e+02, 3.062e+02],
       [5.000e-01, 0.000e+00, 2.389e+02, 6.000e+02, 1.681e+02],
       [6.000e-01, 0.000e+00, 4.283e+02, 6.000e+02, 7.020e+01],
       [6.000e-01, 0.000e+00, 1.333e+02, 6.000e+02, 3.499e+02],
       [6.000e-01, 0.000e+00, 3.050e+02, 6.000e+02, 1.032e+02],
       [6.000e-01, 0.000e+00, 9.200e+01, 6.000e+02, 3.929e+02],
       [6.000e-01, 0.000e+00, 1.859e+02,

In [16]:
from pept.plots import PlotlyGrapher2D

grapher = PlotlyGrapher2D()
grapher.add_lines(lors)
grapher.show()

## 2.2. Voxelise / Pixelise the Lines of Response
The next step is to 'voxelise' the LoRs - that is, to create a grid of square or rectangular cells, and count the number of lines passing through each. For simplicity and ease of visualisation, we work here in 2D, but the real algorithm of course works in 3D.

In the code below you can set the resolution - i.e. pixel size - by varying the pair of values corresponding to the `resolution` parameter. Typing, for example, (20,20) will give a 20 by 20 grid of pixels.

Try some different values for the `resolution` parameter. As discussed in the main text, larger values - i.e. a higher resolution, smaller pixels - will allow the centroid of our particle to be determined more precisely. However, you may notice that as you increase these values your processor may take longer in the calculation and plotting steps below. Now imagine this is just one of many thousands of locations, and you may get an idea of the issue of using an overly-fine grid. As discussed in the main text, when using codes relying on voxelisation, there is a fine balance to be made between precision and compuational efficiency.

In [17]:
resolution = (20, 20)
pixels = pept.Pixels.from_lines(lors, resolution)

pixels

Initialised Pixels class in 0.0005259513854980469 s.


Class instance that inherits from `pept.Pixels`.
Type:
<class 'pept.base.pixel_data.Pixels'>

Attributes
----------
[[ 0.  2.  3.  7.  7.  7.  5.  4.  2. 10.  5.  2.  3.  6.  5.  0.  0.  0.
   0.  0.]
 [ 0.  0.  2.  3.  8.  8.  7.  4.  4.  9.  3.  2.  3.  6.  2.  0.  0.  0.
   0.  0.]
 [ 0.  0.  2.  4.  8. 10.  7.  3.  5.  8.  3.  2.  8.  6.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  3.  4. 14.  8.  4.  7.  8.  5.  3.  7.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  2.  3. 13. 11.  6.  7.  6.  5.  8.  6.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  3.  8. 13.  8.  9.  7.  7.  8.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  5. 11. 13. 10.  6.  8.  5.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  1.  5. 10. 18. 10. 10.  7.  3.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  1.  2.  6. 19. 14. 10.  4.  4.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  2.  5. 19. 23.  8.  3.  2.  1.  0.  0.  0.  0.  0.
   0.  0.]
 [ 0.  0.  0.  1.  2.  2.  4. 18. 

In [18]:
from pept.plots import PlotlyGrapher2D

grapher = PlotlyGrapher2D()

grapher.add_pixels(pixels)
grapher.add_lines(lors, color = "red")

grapher.show()

## 2.3. Subtract Convolved Matrix and Blur

Having created a grid of pixels, we can now, in essence, treat these just like a digital image, as discussed in the main text. 

In the first step below, we are in essence applying a simple smoothing filter (namely a [mean filter](https://homepages.inf.ed.ac.uk/rbf/HIPR2/mean.htm#:~:text=The%20idea%20of%20mean%20filtering,are%20unrepresentative%20of%20their%20surroundings)). Try changing the degree of smoothing exhibited by altering the the values in the line `kernel = np.ones((3, 3))` - higher values will apply the mean filter over a wider area, giving a more significant smoothing effect, and vice versa. 

The smoothed image is then subtracted from the original, 'un-smoothed' image to remove background noise, as can be seen in the images at the end of this section.

In step 2, a [Gaussian blur](https://homepages.inf.ed.ac.uk/rbf/HIPR2/gsmooth.htm) is then applied to the resultant image to allow the subsequent clustering routine to work more effectively. 
Changing the value of `sigma` will alter the width of the Gaussian, i.e. increasing or decreasing the extent of blurring. Try some different values and see what influence they have on the images output. 


In [19]:
from scipy.signal import convolve2d

kernel = np.ones((3, 3))
kernel /= np.sum(kernel)

convolved = convolve2d(pixels, kernel, mode = "same")
pixels_sub = pixels - convolved
pixels_sub[pixels_sub < 0] = 0.

In [20]:
shape = (3, 3)
x, y = np.meshgrid(np.linspace(-1, 1, shape[0]), np.linspace(-1, 1, shape[1]))
d = np.sqrt(x * x + y * y)

sigma = 0.7
mu = 0.0

gaussian_kernel = 1 / sigma / np.sqrt(2 * np.pi) * np.exp(-0.5 * (d - mu)**2 / sigma**2)

pixels_blurred = convolve2d(pixels_sub, gaussian_kernel, mode = "same")
pixels_blurred = pept.Pixels(pixels_blurred, pixels.xlim, pixels.ylim)

pixels_blurred

Class instance that inherits from `pept.Pixels`.
Type:
<class 'pept.base.pixel_data.Pixels'>

Attributes
----------
[[2.51075633e-01 9.24816462e-01 1.63754425e+00 2.92519697e+00
  2.95839584e+00 2.34525014e+00 1.56455517e+00 9.27310223e-01
  1.79255059e+00 4.65943042e+00 2.45083841e+00 4.33676094e-01
  1.10195615e+00 2.88635362e+00 2.43176375e+00 5.93451497e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [9.04996568e-02 3.66257015e-01 6.98003916e-01 1.41380794e+00
  2.09295529e+00 1.92577041e+00 1.23564959e+00 4.93123452e-01
  1.44617186e+00 3.89908430e+00 1.68345025e+00 4.44271043e-01
  1.67496748e+00 2.65693845e+00 1.35541668e+00 2.13908280e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.13002303e-02 3.07174043e-01 5.51296622e-01
  1.71661437e+00 2.59925572e+00 9.83845183e-01 1.39863106e-01
  1.07328875e+00 2.48178148e+00 8.67863228e-01 9.88103570e-01
  3.36688222e+00 2.72361121e+00 6.25559890e-01 0.00000000e+00
  0.00000000e+

In [21]:
grapher = PlotlyGrapher2D(
    cols = 3,
    subplot_titles = ["Original Pixellised LoRs", "Subtracted Convolution", "Gaussian Blur"],
)

grapher.add_pixels(pixels)
grapher.add_pixels(pixels_sub, col = 2)
grapher.add_pixels(pixels_blurred, col = 3)

grapher.show()

## 2.4. Extract Voxel / Pixel Peaks

Finally, from the blurred and subtracted image we extract the peaks which (if our parameters are correct) will represent particle positions.

Specifically, we search for any peaks lying in the upper `r_percent`-th percentile of pixel values (i.e. line densities) - for example, if `r_percent = 10`, we consider only pixels whose values lie in the top 10%. 

In the case of multiple particles, we are not only looking for one global maximum, but multiple local maxima. In this case, we look for peaks within individual 'regions' of size `(2 * w_pixels + 1)`.

Try varying the size of `r_percent` and `w_pixels` and see the effect it has on your results. You will notice for the default settings that the code gives two locations - one true, and one false. You will find that this peak can be removed both by adjusting `w_pixels` _and_ by adjusting `r_percent`. Give some thought as to which of these options would be most effective in different real-world situations. 

In [22]:
# Extract peaks in the upper `r_percent`-th percentile, which also have the maximum value in a
# pixel square of length (2 * `w_pixels` + 1)
r_percent = 10
w_pixels = 3

r_upper_quantile = np.quantile(pixels_blurred, 1 - r_percent/100)
likely_indices = np.argwhere(pixels_blurred > r_upper_quantile)

peaks = []
for ix, iy in likely_indices:
    # Vicinity pixel square indices, truncated at the grid's edges
    min_ix = max(ix - 2 * w_pixels, 0)
    max_ix = min(ix + 2 * w_pixels + 1, pixels_blurred.shape[0])

    min_iy = max(iy - 2 * w_pixels, 0)
    max_iy = min(iy + 2 * w_pixels + 1, pixels_blurred.shape[1])

    # If the current pixel at (ix, iy) is the maximum in the vicinity square, it is a peak
    square = pixels_blurred[min_ix:max_ix, min_iy:max_iy]
    if pixels_blurred[ix, iy] == square.max():
        peaks.append((ix, iy))

peaks = np.array(peaks)

# Transform peaks from pixel indices to physical coordinates
origin = np.array([pixels.xlim[0], pixels.ylim[0]])
positions = origin + peaks * pixels.pixel_size + 0.5 * pixels.pixel_size

# Insert the tracer position's timestamp at column 0
positions = np.insert(positions, 0, lors[:, 0].mean(), axis = 1)
positions

array([[  0.938,  15.   , 285.   ],
       [  0.938, 315.   , 255.   ]])

In [23]:
grapher = PlotlyGrapher2D(cols = 2)

grapher.add_pixels(pixels_blurred)
grapher.add_points(positions, size = 10, color = "red")

grapher.add_lines(lors, col = 2)
grapher.add_points(positions, size = 10, color = "red", col = 2)

grapher.show()

# 3. Complete FPI Code

In [24]:
# Read in a sample of experimental PEPT data from an online repository into a NumPy array
import numpy as np
import pept

# Skip the file header's first 15 lines, then read in 50 LoRs
lors_raw = pept.utilities.read_csv(
    "https://raw.githubusercontent.com/uob-positron-imaging-centre/example_data/master/sample_1p_fluidised_bed.csv",
    skiprows = 15,
    nrows = 80_000,
)

# Insert columns for the z-coordinates
head_separation = 600

lors_raw = np.insert(lors_raw, 3, 0, axis = 1)
lors_raw = np.insert(lors_raw, 6, head_separation, axis = 1)

# Project the 3D lines onto the YZ plane for ease of analysis - i.e. select only columns
# [time, y1, z1, y2, z2] and flip columns to get [time, x1, y1, x2, y2]
lors = np.array(lors_raw[:, [0, 3, 2, 6, 5]])

# Print the line of response (LoR) data
lors

array([[0.0000e+00, 0.0000e+00, 1.6870e+02, 6.0000e+02, 1.4280e+02],
       [1.0000e-01, 0.0000e+00, 1.6760e+02, 6.0000e+02, 3.1390e+02],
       [1.0000e-01, 0.0000e+00, 4.1000e+02, 6.0000e+02, 2.4010e+02],
       ...,
       [2.6443e+03, 0.0000e+00, 2.2480e+02, 6.0000e+02, 3.4280e+02],
       [2.6444e+03, 0.0000e+00, 1.3690e+02, 6.0000e+02, 1.4750e+02],
       [2.6444e+03, 0.0000e+00, 5.4220e+02, 6.0000e+02, 5.5280e+02]])

In [25]:
# Use the Feature Point Identification (FPI) algorithm to track moving tracer
from scipy.signal import convolve2d

# User-defined FPI settings
resolution = (100, 100)     # Number of pixels in the LoR pixellisation step
shape = (3, 3)              # Size of the convolution / blur kernels
sigma = 0.7                 # Standard deviation in the Gaussian blur kernel
r_percent = 0.05            # Select the upper `r_percent` pixel values
w_pixels = 3                # Square of length (2 * w + 1) for selecting pixel peak

sample_size = 100           # Number of LoRs in a sample

sample_start = 0
positions = []

while sample_start + sample_size < len(lors):
    sample = lors[sample_start:sample_start + sample_size]

    # Pixellise sample of LoRs
    pixels = pept.Pixels.from_lines(sample, resolution, verbose = False)

    xlim = pixels.xlim
    ylim = pixels.ylim

    # Subtract convolved matrix and threshold pixel values to zero
    kernel = np.ones((3, 3))
    kernel /= np.sum(kernel)

    convolved = convolve2d(pixels, kernel, mode = "same")
    pixels -= convolved
    pixels[pixels < 0] = 0.

    # Apply 2D Gaussian blur
    x, y = np.meshgrid(np.linspace(-1, 1, shape[0]), np.linspace(-1, 1, shape[1]))
    d = np.sqrt(x * x + y * y)

    mu = 0.0

    gaussian_kernel = 1 / sigma / np.sqrt(2 * np.pi) * np.exp(-0.5 * (d - mu)**2 / sigma**2)

    pixels = convolve2d(pixels, gaussian_kernel, mode = "same")
    pixels = pept.Pixels(pixels, xlim, ylim)

    # Extract peaks in the upper `r_percent`-th percentile, which also have the maximum value in a
    # pixel square of length (2 * `w_pixels` + 1)
    r_upper_quantile = np.quantile(pixels, 1 - r_percent/100)
    likely_indices = np.argwhere(pixels > r_upper_quantile)

    peaks = []
    for ix, iy in likely_indices:
        # Vicinity pixel square indices, truncated at the grid's edges
        min_ix = max(ix - 2 * w_pixels, 0)
        max_ix = min(ix + 2 * w_pixels + 1, pixels.shape[0])

        min_iy = max(iy - 2 * w_pixels, 0)
        max_iy = min(iy + 2 * w_pixels + 1, pixels.shape[1])

        # If the current pixel at (ix, iy) is the maximum in the vicinity square, it is a peak
        square = pixels[min_ix:max_ix, min_iy:max_iy]
        if pixels[ix, iy] == square.max():
            peaks.append((ix, iy))

    peaks = np.array(peaks)

    # Transform peaks from pixel indices to physical coordinates
    origin = np.array([pixels.xlim[0], pixels.ylim[0]])
    centres = origin + peaks * pixels.pixel_size + 0.5 * pixels.pixel_size

    # Insert the tracer position's timestamp at column 0 and save them
    centres = np.insert(centres, 0, sample[:, 0].mean(), axis = 1)
    positions.extend(centres)

    sample_start += sample_size

positions = np.array(positions)
positions

array([[1.931000e+00, 3.210000e+02, 2.430000e+02],
       [5.878000e+00, 3.150000e+02, 2.430000e+02],
       [9.966000e+00, 3.150000e+02, 2.430000e+02],
       ...,
       [2.633128e+03, 3.030000e+02, 2.910000e+02],
       [2.636749e+03, 3.030000e+02, 2.910000e+02],
       [2.639971e+03, 3.090000e+02, 2.910000e+02]])

In [26]:
grapher = PlotlyGrapher2D(cols = 3)

grapher.add_lines(lors[:400])
grapher.add_pixels(pixels, col = 2)
grapher.add_points(positions, col = 3)

grapher.show()